In [1]:
import sympy as sym
import symnum
import symnum.numpy as snp
import numpy as np
import multiprocessing

In [2]:
y = snp.array([1., 0.])
σ = 0.1

def forward_func(θ):
    return snp.array([[1., -0.5], [-2., 3.]]) @ snp.array(
        [snp.cos(-θ[1]**2 + 3 * θ[0]), snp.sin(θ[0] - 1)])

def neg_log_dens(θ):
    return (
        0.5 * snp.sum((y - forward_func(θ))**2 / σ**2) + 
        0.5 * snp.sum(θ**2))

## Symbolic derivatives

In [3]:
θ = symnum.array.named_array('θ', 2)

In [4]:
forward_func(θ)

[-0.5*sin(θ[0] - 1) + 1.0*cos(3*θ[0] - θ[1]**2), 3.0*sin(θ[0] - 1) - 2.0*cos(3*θ[0] - θ[1]**2)]

In [5]:
symnum.sympy_jacobian(forward_func)(θ)

[[-3.0*sin(3*θ[0] - θ[1]**2) - 0.5*cos(θ[0] - 1), 2.0*θ[1]*sin(3*θ[0] - θ[1]**2)], [6.0*sin(3*θ[0] - θ[1]**2) + 3.0*cos(θ[0] - 1), -4.0*θ[1]*sin(3*θ[0] - θ[1]**2)]]

In [6]:
neg_log_dens(θ)

0.5*θ[0]**2 + 0.5*θ[1]**2 + 450.0*(-sin(θ[0] - 1) + 0.666666666666667*cos(3*θ[0] - θ[1]**2))**2 + 50.0*(0.5*sin(θ[0] - 1) - 1.0*cos(3*θ[0] - θ[1]**2) + 1.0)**2

In [7]:
symnum.sympy_grad(neg_log_dens)(θ)

[1.0*θ[0] + 450.0*(-sin(θ[0] - 1) + 0.666666666666667*cos(3*θ[0] - θ[1]**2))*(-4.0*sin(3*θ[0] - θ[1]**2) - 2*cos(θ[0] - 1)) + 50.0*(6.0*sin(3*θ[0] - θ[1]**2) + cos(θ[0] - 1))*(0.5*sin(θ[0] - 1) - 1.0*cos(3*θ[0] - θ[1]**2) + 1.0), 1200.0*θ[1]*(-sin(θ[0] - 1) + 0.666666666666667*cos(3*θ[0] - θ[1]**2))*sin(3*θ[0] - θ[1]**2) - 200.0*θ[1]*(0.5*sin(θ[0] - 1) - 1.0*cos(3*θ[0] - θ[1]**2) + 1.0)*sin(3*θ[0] - θ[1]**2) + 1.0*θ[1]]

In [8]:
symnum.sympy_hessian(neg_log_dens)(θ)

[[(-450.0*sin(θ[0] - 1) + 300.0*cos(3*θ[0] - θ[1]**2))*(2*sin(θ[0] - 1) - 12.0*cos(3*θ[0] - θ[1]**2)) + (-50.0*sin(θ[0] - 1) + 900.0*cos(3*θ[0] - θ[1]**2))*(0.5*sin(θ[0] - 1) - 1.0*cos(3*θ[0] - θ[1]**2) + 1.0) + (-900.0*sin(3*θ[0] - θ[1]**2) - 450.0*cos(θ[0] - 1))*(-4.0*sin(3*θ[0] - θ[1]**2) - 2*cos(θ[0] - 1)) + (3.0*sin(3*θ[0] - θ[1]**2) + 0.5*cos(θ[0] - 1))*(300.0*sin(3*θ[0] - θ[1]**2) + 50.0*cos(θ[0] - 1)) + 1.0, 3600.0*θ[1]*(-sin(θ[0] - 1) + 0.666666666666667*cos(3*θ[0] - θ[1]**2))*cos(3*θ[0] - θ[1]**2) + 1200.0*θ[1]*(-2.0*sin(3*θ[0] - θ[1]**2) - cos(θ[0] - 1))*sin(3*θ[0] - θ[1]**2) - 200.0*θ[1]*(3.0*sin(3*θ[0] - θ[1]**2) + 0.5*cos(θ[0] - 1))*sin(3*θ[0] - θ[1]**2) - 600.0*θ[1]*(0.5*sin(θ[0] - 1) - 1.0*cos(3*θ[0] - θ[1]**2) + 1.0)*cos(3*θ[0] - θ[1]**2)], [8.0*θ[1]*(-450.0*sin(θ[0] - 1) + 300.0*cos(3*θ[0] - θ[1]**2))*cos(3*θ[0] - θ[1]**2) + 600.0*θ[1]*(-4.0*sin(3*θ[0] - θ[1]**2) - 2*cos(θ[0] - 1))*sin(3*θ[0] - θ[1]**2) - 2.0*θ[1]*(300.0*sin(3*θ[0] - θ[1]**2) + 50.0*cos(θ[0] - 1))*sin

## NumPy function generation

In [9]:
θ_arr = np.array([0.2, 1.1])

In [10]:
symnum.numpify(2)(forward_func)(θ_arr)

array([ 1.17832606, -3.79136431])

In [11]:
symnum.numpify(2)(neg_log_dens)(θ_arr)

720.9371751890193

In [12]:
grad_neg_log_dens = symnum.numpy_grad(neg_log_dens, 2)
grad_neg_log_dens(θ_arr)

array([ 535.36397107, -977.0322501 ])

In [13]:
hess_neg_log_dens = symnum.numpy_hessian(neg_log_dens, 2)
hess_neg_log_dens(θ_arr)

array([[-6177.30148469,  3686.23777978],
       [ 3686.23777978, -3172.9077026 ]])

In [14]:
mtp_neg_log_dens = symnum.numpy_mtp(neg_log_dens, 2)
mtp_neg_log_dens(θ_arr)(hess_neg_log_dens(θ_arr))

array([ 3.03591481e+08, -2.17976284e+08])

In [15]:
jacob_forward_func = symnum.numpy_jacobian(forward_func, 2)
jacob_forward_func(θ_arr)

array([[ 1.37024903, -1.26030841],
       [-1.34708463,  2.52061682]])

In [16]:
mhp_forward_func = symnum.numpy_mhp(forward_func, 2)
mhp_forward_func(θ_arr)(jacob_forward_func(θ_arr))

array([-67.46233454,  54.20591147])

## Compatibility with `multiprocessing`

In [17]:
pool = multiprocessing.Pool(4)

In [18]:
pool.map(grad_neg_log_dens, [θ_arr + 0.1 * i for i in range(8)])

[array([ 535.36397107, -977.0322501 ]),
 array([ 309.46173134, -921.2782253 ]),
 array([ 137.12445164, -865.25332766]),
 array([  22.61966799, -818.58457381]),
 array([ -36.90904075, -785.80345863]),
 array([ -49.75261632, -765.70482534]),
 array([ -28.37869342, -750.84259881]),
 array([  11.35729628, -726.98047844])]